In [1]:
import doc_objects
import time
import config

# doc_objects.send_msg(723253749, 'Старт скрипта.')

g1 = doc_objects.Genset('/dev/ttyS0', 1)
g2 = doc_objects.Genset('/dev/ttyS0', 2)
g3 = doc_objects.Genset('/dev/ttyS0', 3)
g4 = doc_objects.Genset('/dev/ttyS0', 4)
g5 = doc_objects.Genset('/dev/ttyS0', 5)

doc = (g1, g2, g3, g4, g5)

In [8]:
!pip3 freeze

asn1crypto==0.24.0
bleach==3.1.2
certifi==2018.8.24
chardet==3.0.4
cryptography==2.6.1
decorator==4.3.0
defusedxml==0.5.0
entrypoints==0.3
html5lib==1.0.1
idna==2.6
ipykernel==4.9.0
ipython==5.8.0
ipython-genutils==0.2.0
ipywidgets==6.0.0
Jinja2==2.10
jsonschema==2.6.0
jupyter-client==5.2.3
jupyter-core==4.4.0
keyring==17.1.1
keyrings.alt==3.1.1
MarkupSafe==1.1.0
minimalmodbus==1.0.2
mistune==0.8.4
nbconvert==5.4.0
nbformat==4.4.0
notebook==5.7.8
pandocfilters==1.4.2
pexpect==4.6.0
pickleshare==0.7.5
prometheus-client==0.6.0
prompt-toolkit==1.0.15
pycrypto==2.6.1
Pygments==2.3.1
PyGObject==3.30.4
pyserial==3.5
pyTelegramBotAPI==3.7.6
python-apt==1.8.4.3
python-dateutil==2.7.3
pyxdg==0.25
pyzmq==17.1.2
requests==2.21.0
SecretStorage==2.3.1
Send2Trash==1.5.0
simplegeneric==0.8.1
six==1.12.0
ssh-import-id==5.7
terminado==0.8.1
testpath==0.4.2
tornado==5.1.1
traitlets==4.3.2
urllib3==1.24.1
wcwidth==0.1.7
webencodings==0.5.1
widgetsnbextension==2.0.0


In [3]:
g1.modbus_table

{2: {'Name': 'BIN',
  'Name_in_bd': 'BIN',
  'Dim': '',
  'Type': 'Binary#1',
  'Len': 2,
  'Dec': '-',
  'Min': '-',
  'Max': '-',
  'Group': 'Bin inputs CU',
  'Prev_val': None,
  'Curr_val': None},
 3: {'Name': 'BIN-1',
  'Name_in_bd': 'BIN_1',
  'Dim': '',
  'Type': 'Binary#2',
  'Len': 1,
  'Dec': '-',
  'Min': '-',
  'Max': '-',
  'Group': 'Binary Inputs',
  'Prev_val': None,
  'Curr_val': None},
 4: {'Name': 'BIN-2',
  'Name_in_bd': 'BIN_2',
  'Dim': '',
  'Type': 'Binary#3',
  'Len': 1,
  'Dec': '-',
  'Min': '-',
  'Max': '-',
  'Group': 'Binary Inputs',
  'Prev_val': None,
  'Curr_val': None},
 5: {'Name': 'BIN-3',
  'Name_in_bd': 'BIN_3',
  'Dim': '',
  'Type': 'Binary#4',
  'Len': 1,
  'Dec': '-',
  'Min': '-',
  'Max': '-',
  'Group': 'Binary Inputs',
  'Prev_val': None,
  'Curr_val': None},
 6: {'Name': 'BIN-4',
  'Name_in_bd': 'BIN_4',
  'Dim': '',
  'Type': 'Binary#5',
  'Len': 1,
  'Dec': '-',
  'Min': '-',
  'Max': '-',
  'Group': 'Binary Inputs',
  'Prev_val': None,


In [22]:
start_time = time.time()

print(time.time() - start_time)

In [7]:
import sqlite3,  config
conn = sqlite3.connect(config.raspi_bd)
curs = conn.cursor()
ins = f'SELECT * FROM {config.table_regular_values} WHERE date_time=(SELECT max(date_time) FROM {config.table_regular_values})'
curs.execute(ins)
apower = curs.fetchall()
curs.close()
conn.close()
apower


[('2021-02-11 18:55:18',
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,
  None,

In [9]:
import minimalmodbus
import config
import logging
import telebot
import sqlite3
import datetime

def init_bd(doc):
    print('start init BD')
    conn = sqlite3.connect(config.raspi_bd)
    curs = conn.cursor()
    try:
        ins = f'SELECT * FROM {config.table_regular_values}'
        curs.execute(ins)
        print('успешная попытка чтения из БД, создание не требуется')
    except Exception as e:
        print(f'создание таблицы: {config.table_regular_values}')
        logging.info(f'Создание таблицы значений: {config.table_regular_values}')
        ins = f'CREATE TABLE {config.table_regular_values} (date_time VARCHAR(20) PRIMARY KEY'
        for g in doc:
            for k, v in {k: v for k, v in g.modbus_table.items() if v['Group'] in config.regular_values}.items():
                if v['Type'] == 'Integer' or v['Type'] == 'Unsigned':
                    type_field = 'INT'
                else:
                    type_field = 'CHAR (20)'
                ins += f", Genset{g.address}_{v['Name_in_bd']} {type_field}"
        ins += f')'
        curs.execute(ins)
        conn.commit()
        print(f'Таблица {config.table_regular_values} создана.')
        logging.info(f'Таблица {config.table_regular_values} создана.')
        
    curs.close()
    conn.close()
    
init_bd(doc)

start init BD
создание таблицы: g_val
Таблица g_val создана.


In [16]:
!rm debug_log

'g_val'

In [17]:
!python3 main.py

Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
5.9 seconds
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
5.6 seconds
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
6.5 seconds
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
5.9 seconds
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
6.8 seconds
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
5.9 seconds
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
5.9 seconds
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
5.8 seconds
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
6.2 seconds
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
5.8 seconds
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main
Цикл for в main


In [9]:
import sqlite3
import config

conn = sqlite3.connect(config.raspi_bd)
curs = conn.cursor()
ins = f'SELECT Oil_Pressure FROM {config.table_regular_values}'
curs.execute(ins)
curs.fetchall()

OperationalError: no such column: Oil_Pressure

In [31]:
import sqlite3

table_name = 'g_val'

conn = sqlite3.connect('bd_test.bd')
curs = conn.cursor()

ins = f'CREATE TABLE {table_name} (date_time VARCHAR(20) PRIMARY KEY'
for g in doc:
    for k, v in {k: v for k, v in g.modbus_table.items() if v['Group'] in config.regular_values}.items():
        if v['Type'] == 'Integer' or v['Type'] == 'Unsigned':
            type_field = 'INT'
        else:
            type_field = 'CHAR (20)'
        ins += f", Genset{g.address}_{v['Name_in_bd']} {type_field}"
ins += f')'
print(ins)
curs.execute(ins)
conn.commit()
curs.close()
conn.close()


CREATE TABLE g_val (date_time VARCHAR(20) PRIMARY KEY, Genset1_BIN CHAR (20), Genset1_BIN_1 CHAR (20), Genset1_BIN_2 CHAR (20), Genset1_BIN_3 CHAR (20), Genset1_BIN_4 CHAR (20), Genset1_BIN_5 CHAR (20), Genset1_BIN_6 CHAR (20), Genset1_BIN_7 CHAR (20), Genset1_BIN_8 CHAR (20), Genset1_BOUT CHAR (20), Genset1_Ubat INT, Genset1_CPU_temp INT, Genset1_Engine_Temp INT, Genset1_Oil_Pressure INT, Genset1_Oil_Temp INT, Genset1_Manifold_Temp INT, Genset1_Manifold_Press INT, Genset1_Actual_Timing INT, Genset1_Desired_Timing INT, Genset1_Detonation INT, Genset1_Cylinder_1 INT, Genset1_Cylinder_2 INT, Genset1_Cylinder_3 INT, Genset1_Cylinder_4 INT, Genset1_Cylinder_5 INT, Genset1_Cylinder_6 INT, Genset1_Cylinder_7 INT, Genset1_Cylinder_8 INT, Genset1_Cylinder_9 INT, Genset1_Cylinder_10 INT, Genset1_Cylinder_11 INT, Genset1_Cylinder_12 INT, Genset1_Cylinder_13 INT, Genset1_Cylinder_14 INT, Genset1_Cylinder_15 INT, Genset1_Cylinder_16 INT, Genset1_JW_Flap_Real INT, Genset1_AIN_4_2 INT, Genset1_AC_In

In [3]:
import datetime
import sqlite3

table_name = 'g_val'

while 1:
    data = [datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')]

    ins = f'INSERT INTO {table_name} (date_time'
    ins_val = ' VALUES(?'
    for g in doc:
        for k, v in {k: v for k, v in g.modbus_table.items() if v['Group'] in config.regular_values}.items():
            ins += f", Genset{g.address}_{v['Name_in_bd']}"
            ins_val += ', ?'
            data.append(v['Curr_val'])
    ins += ') ' + ins_val + ')'
    conn = sqlite3.connect('bd_test.bd')
    curs = conn.cursor()
    curs.execute(ins, data)
    conn.commit()
    curs.close()
    conn.close()

0.5942397117614746
0.2980923652648926
0.8521709442138672
0.5733461380004883
0.5845241546630859
0.48847270011901855
0.5934090614318848
0.5679819583892822
0.349395751953125
0.11561036109924316
1.1162545680999756
0.5436370372772217
0.8439030647277832
0.7837557792663574
0.5195655822753906
0.5525031089782715
0.8202333450317383
0.5552027225494385
0.5974178314208984
0.5145959854125977
1.458832025527954
0.5394353866577148
0.6055090427398682
0.12406039237976074
0.8187446594238281
0.3805849552154541
0.6126258373260498
0.787024736404419
0.613178014755249
0.16183757781982422
0.8566813468933105
0.3748505115509033
0.7378189563751221
0.12424945831298828
0.5707008838653564
0.11289501190185547
0.8467257022857666
0.5274579524993896
0.5767834186553955
0.6072542667388916
0.5846178531646729
0.4164113998413086
0.8531076908111572
0.30951857566833496
0.5796148777008057
0.3914492130279541
1.2846033573150635
0.5366611480712891
0.6256289482116699
0.5617005825042725
0.6479995250701904
0.11356472969055176
0.560712

0.3459498882293701
0.602057933807373
0.5479917526245117
0.8154008388519287
0.7660667896270752
0.8269984722137451
0.5477263927459717
0.5639548301696777
0.5574522018432617
0.8163878917694092
0.12104105949401855
0.3516676425933838
0.3596310615539551
0.608405590057373
0.535285234451294
0.3987545967102051
0.36334919929504395
0.8955483436584473
0.4213676452636719
0.3423454761505127
0.3589913845062256
0.8402724266052246
0.5342025756835938
0.5997538566589355
0.3643820285797119
0.8795990943908691
0.591850996017456
0.5946860313415527
0.08351898193359375
1.254885196685791
0.939305305480957
0.8451845645904541
0.35920047760009766
0.8830931186676025
0.8221051692962646
0.6379215717315674
0.7774808406829834
0.6174137592315674
0.3727304935455322
0.6290509700775146
0.35962367057800293
0.6221864223480225
0.15477299690246582
0.6059155464172363
0.7665162086486816
0.8116171360015869
0.3459751605987549
0.6071505546569824
0.7831153869628906
0.8277623653411865
0.5785262584686279
0.7850263118743896
0.3359515666

KeyboardInterrupt: 

In [ ]:
    data_list = [datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')]
    ins = f'INSERT INTO {table_name} (date_time'
    ins_val = ' VALUES(?'
    for g in g_list:
        g._prev_data = [v[0] for k, v in g._device_values.items()]  # Сохраняю предыдущие данные
        g.read_values()                                             # Читаю свежие данные
                
        for k, v in g._device_values.items():
    # Собираю строку для записи в БД ------------------------------------------------------------------------------------ 
            ins += f', {g.name}_{k}'
            ins_val += ', ?'
            data_list.append(v[0])
    ins += ') ' + ins_val + ')'


In [14]:
len({k: v for k, v in g1.modbus_table.items() if v['Group'] in config.regular_values})

100

In [2]:
start_time = time.time()
for g in doc:
    g.get_update()
print((f'{time.time() - start_time:.1f} seconds'))

13.6 seconds


In [4]:
set([v['Type'] for k, v in g4.modbus_table.items()])

{'Binary#1',
 'Binary#10',
 'Binary#11',
 'Binary#2',
 'Binary#3',
 'Binary#4',
 'Binary#42',
 'Binary#43',
 'Binary#44',
 'Binary#45',
 'Binary#5',
 'Binary#6',
 'Binary#7',
 'Binary#8',
 'Binary#9',
 'Integer',
 'Unsigned'}

In [7]:
[(v['Name'], v['Curr_val'], v['Dim']) for k, v in g2.modbus_table.items()]

[('BIN', '0b10000001101010', ''),
 ('BIN-1', '0b00000011100010', ''),
 ('BIN-2', '0b00000010000010', ''),
 ('BIN-3', '0b00000000001111', ''),
 ('BIN-4', '0b00000000000000', ''),
 ('BIN-5', '0b00000001010101', ''),
 ('BIN-6', '0b00000000101001', ''),
 ('BIN-7', '0b00000001010100', ''),
 ('BIN-8', '0b00000000001010', ''),
 ('BOUT', '0b01000001000000', ''),
 ('Ubat', 25.9, 'V'),
 ('CPU temp', 26.8, '°C'),
 ('Engine Temp', 48.0, '°C'),
 ('Oil Pressure', 0.0, 'bar'),
 ('Oil Temp', 29.0, '°C'),
 ('Manifold Temp', 31.0, '°C'),
 ('Manifold Press', 2.2, 'bar'),
 ('Actual Timing', 0.0, '°'),
 ('Desired Timing', 0.0, '°'),
 ('Detonation', 0.0, ''),
 ('Cylinder 1', 36.0, '°C'),
 ('Cylinder 2', 36.0, '°C'),
 ('Cylinder 3', 37.0, '°C'),
 ('Cylinder 4', 35.0, '°C'),
 ('Cylinder 5', 36.0, '°C'),
 ('Cylinder 6', 36.0, '°C'),
 ('Cylinder 7', 37.0, '°C'),
 ('Cylinder 8', 35.0, '°C'),
 ('Cylinder 9', 37.0, '°C'),
 ('Cylinder 10', 35.0, '°C'),
 ('Cylinder 11', 35.0, '°C'),
 ('Cylinder 12', 35.0, '°C'),
 ('

In [9]:
g = g4

def _formating_register(adr, reg):
    # Если бинарное число, то возвращаю в двоичном виде
    if g.modbus_table[adr]['Type'].lower()[:3] == 'bin':
        return format(reg, '#016b')
    # Отрицательное число определяю по косвенным признакам
    elif (not g.modbus_table[adr]['Min'].isdigit()) and len(g.modbus_table[adr]['Min'])-1 and g.modbus_table[adr]['Min'][-1]!='*' and reg&0b1000000000000000:
        reg = reg - 65535
    # Десятичная точка
    dec = g.modbus_table[adr]['Dec']
    if dec!='-':
        reg = reg/10**int(dec)
    return reg
    
    
    
def get_update():
    chank_arr = g.get_chunk_intervals(list(g.modbus_table.keys()))
    for chank in chank_arr:
        regs = g.read_mb_registers(chank[0], chank[1])
        if regs:
            for c in range(len(regs)):
                adr = chank[0] + c
                g.modbus_table[adr]['Prev_val'] = g.modbus_table[adr]['Curr_val']
                g.modbus_table[adr]['Curr_val'] = _formating_register(adr, regs[c])
    return None

start_time = time.time()
get_update()
print((f'{time.time() - start_time:.1f} seconds'))

4.5 seconds


In [31]:
format(3, '#016b')
g4.modbus_table[5]['Type'].lower()[:3] == 'bin'

'bin'

У меня есть массив данных, вся таблица регистров, но мне они все не нужны. Есть критерии для опроса
Надо опрашивать, только то что надо, что не надо опрашивать не надо
long опрашиваю отдельно, их не надо в пакетный опрос включать
Должны быть две группы параметров, для регулярного внесения в БД и для внесения по изменению (это статические параметры)

In [12]:
config.values_if_changed + config.regular_values

('Analog protect',
 'Basic settings',
 'Comms settings',
 'Engine params',
 'Engine protect',
 'Gener protect',
 'Load shedding',
 'Mains protect',
 'PLC',
 'ProcessControl',
 'Pwr management',
 'Sync/Load ctrl',
 'Volt/PF ctrl',
 'Analog CU',
 'AnalogInputs 1',
 'AnalogInputs 2',
 'AnalogInputs 3',
 'AnalogInputs 4',
 'AnalogInputs 5',
 'Bus values',
 'Engine values',
 'Gener values',
 'Bin inputs CU',
 'Bin outputs CU',
 'Binary Inputs',
 'Binary Outputs')

In [17]:
import config
len({(v['Name'], v['Curr_val']) for k, v in g1.modbus_table.items() if v['Group'] in (config.values_if_changed + config.regular_values) and v['Name'] not in (config.val_ignore_list)})

443

In [9]:
start_time = time.time()

for adr in g1.modbus_table:
    # Проверяю условия:
    # если 'Name' в списке игнорируемых, то пропускаю цикл
    if g1.modbus_table[adr]['Name'] in config.val_ignore_list:
        continue
    # если 'Name' в списке для read_long, то читаю функцией read_long
    elif g1.modbus_table[adr]['Name'] in config.var_long_list:
        g1.modbus_table[adr]['Curr_val'] = g1.read_mb_long(adr)
        continue
    # если max<0 то signed
#     elif (not g1.modbus_table[adr]['Min'].isdigit()) and len(g1.modbus_table[adr]['Min'])-1 and g1.modbus_table[adr]['Min'][-1]!='*':
#         g1.modbus_table[adr]['Curr_val'] = int(g1.read_mb_register(adr))-65535
    # если 'Type' is 'bin', то записываю в двоичном виде
    g1.modbus_table[adr]['Curr_val'] = g1.read_mb_register(adr)
    
print((f'{time.time() - start_time:.1f} seconds'))

21.1 seconds


In [2]:
!kill 1132

In [4]:
print(f'ГПГУ{g1.address}, наработка: {g1.read_mb_long(3586)} мч')
print(f'ГПГУ{g2.address}, наработка: {g2.read_mb_long(3586)} мч')
print(f'ГПГУ{g3.address}, наработка: {g3.read_mb_long(3821)} мч')
print(f'ГПГУ{g4.address}, наработка: {g4.read_mb_long(3821)} мч')
print(f'ГПГУ{g5.address}, наработка: {g5.read_mb_long(3821)} мч')

ГПГУ1, наработка: None мч
ГПГУ2, наработка: 8109 мч
ГПГУ3, наработка: None мч
ГПГУ4, наработка: None мч
ГПГУ5, наработка: None мч


ГПГУ1, наработка: 8461 мч
ГПГУ2, наработка: 7930 мч
ГПГУ3, наработка: None мч
ГПГУ4, наработка: [0, 9241] мч
ГПГУ5, наработка: None мч

In [5]:
import telebot
token = '1325955552:AAHQJzvhu-IhcMpsVGX5dUsfXTJ7fFECoJs'

tb = telebot.TeleBot(token)